#### Подготовка Данных

##### EDA

In [9]:
from pathlib import Path
import pandas as pd

file_path = Path.cwd() / "ml" / "data" / "raw" / "IBM HR Data new.csv"
print("cwd:", Path.cwd())
print("try:", file_path)

if not file_path.exists():
    file_path = Path.cwd().parent / "ml" / "data" / "raw" / "IBM HR Data new.csv"
    print("try2:", file_path)

print("exists:", file_path.exists())

df = pd.read_csv(file_path)
df.shape
df.info()

miss = (df.isna().mean() * 100).sort_values(ascending=False)
miss[miss > 0].head(15)

print(f'Over18 {df["Over18"].value_counts(dropna=False)}')
print(f'Over18 uniaue {df["Over18"].unique()}')


df["Attrition"].value_counts(dropna=False).head(20)


cwd: /home/pydev/Desktop/courses/AI-project/notebooks
try: /home/pydev/Desktop/courses/AI-project/notebooks/ml/data/raw/IBM HR Data new.csv
try2: /home/pydev/Desktop/courses/AI-project/ml/data/raw/IBM HR Data new.csv
exists: True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23436 entries, 0 to 23435
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       23433 non-null  float64
 1   Attrition                 23423 non-null  object 
 2   BusinessTravel            23428 non-null  object 
 3   DailyRate                 23424 non-null  float64
 4   Department                23425 non-null  object 
 5   DistanceFromHome          23427 non-null  object 
 6   Education                 23424 non-null  float64
 7   EducationField            23427 non-null  object 
 8   EmployeeCount             23431 non-null  object 
 9   EmployeeNumber            23435 non-null  object 
 

/tmp/ipykernel_23459/1929491347.py:14: DtypeWarning: Columns (5,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Attrition
Current employee         19714
Voluntary Resignation     3709
NaN                         13
Name: count, dtype: int64

##### Data preparation

In [10]:

obj_cols = df.select_dtypes(include="object").columns
num_like = []
for c in obj_cols:
    s = df[c].dropna().astype(str).str.replace(",", ".", regex=False).str.strip()
    share = pd.to_numeric(s, errors="coerce").notna().mean()
    if share > 0.9:
        num_like.append(c)

print("object -> numeric candidates:", num_like)

for c in num_like:
    s = df[c].astype(str).str.replace(",", ".", regex=False).str.strip()
    df[c] = pd.to_numeric(s, errors="coerce")

num_cols = df.select_dtypes(include=["number"]).columns
cat_cols = df.select_dtypes(include="object").columns

df[num_cols] = df[num_cols].fillna(df[num_cols].median(numeric_only=True))

for c in cat_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].mode(dropna=True)[0])

miss_after = (df.isna().mean() * 100).sort_values(ascending=False)
print("top missing after:", miss_after.head(10).to_dict())

df = df.drop(columns=["EmployeeCount", "EmployeeNumber", "Application ID", "Over18"])
df["target"] = (df["Attrition"] == "Voluntary Resignation").astype(int)


object -> numeric candidates: ['DistanceFromHome', 'EmployeeCount', 'EmployeeNumber', 'Application ID', 'HourlyRate', 'JobSatisfaction', 'MonthlyIncome', 'PercentSalaryHike']
top missing after: {'Age': 0.0, 'Attrition': 0.0, 'BusinessTravel': 0.0, 'DailyRate': 0.0, 'Department': 0.0, 'DistanceFromHome': 0.0, 'Education': 0.0, 'EducationField': 0.0, 'EmployeeCount': 0.0, 'EmployeeNumber': 0.0}


##### Export: сохранить processed

In [12]:

out = Path("../ml/data/processed/hr_processed.csv")
out.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out, index=False)

print("saved:", out.resolve())
print("shape:", df.shape)
print("target share:", df["target"].value_counts(normalize=True).to_dict())

saved: /home/pydev/Desktop/courses/AI-project/ml/data/processed/hr_processed.csv
shape: (23436, 34)
target share: {0: 0.8417392046424305, 1: 0.15826079535756954}
